# $\texttt{GiRaFFEfood}$: Initial data for $\texttt{GiRaFFE}$ Aligned Rotator

## Author: Zach Etienne & Patrick Nelson
### Formatting improvements courtesy Brandon Clark

## Introduction: 

This module provides another initial data option for $\texttt{GiRaFFE}$. This is a flat-spacetime test with initial data $$A_{\phi} = \frac{\mu \varpi^2}{r^3},$$ where  $\mu = B_p R^3_{\rm NS} / 2$, $R_{\rm NS}$ is the neutron star radius, and $\varpi = \sqrt{x^2+y^2}$ is the cylindrical radius. We let $A_r = A_\theta = 0$.

Additionally, the drift velocity $v^i = \Omega \textbf{e}_z \times \textbf{r} = [ijk] \Omega \textbf{e}^j_z x^k$, where $[ijk]$ is the Levi-Civita permutation symbol and $\textbf{e}^i_z = (0,0,1)$. See [previous NRPy+ tutorial module](Tutorial-GiRaFFEfood_HO.ipynb) for details on how this is used.

<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 2](#set_aphi): Set the vector $A_{\phi}$ Spherical coordinates 
1. [Step 3](#jacobian) Use the Jacobian matrix to transform the vectors to Cartesian coordinates
1. [Step 4](#vi) Calculate $v^i$
1. [Step 5](#code_validation): Code Validation against GiRaFFEfood_HO/GiRaFFEfood_HO_Aligned_Rotator NRPy+ Module
1. [Step 6](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Here, we will import the NRPy+ core modules and set the reference metric to Cartesian, set commonly used NRPy+ parameters, and set C parameters that will be set from outside the code eventually generated from these expressions. We will also set up a parameter to determine what initial data is set up, although it won't do much yet.

In [1]:
# Step 0: Import the NRPy+ core modules and set the reference metric to Cartesian
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

import reference_metric as rfm
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Step 1a: Set commonly used parameters.
thismodule = "GiRaFFEfood_HO_Aligned_Rotator"
# Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

B_p_aligned_rotator,R_NS_aligned_rotator = par.Cparameters("REAL",thismodule,["B_p_aligned_rotator","R_NS_aligned_rotator"]) # A constant defining the intensity of the magnetic field and the Neutron star Radius


<a id='set_aphi'></a>

# Step 2: Set the vector $A_{\phi}$ in Spherical coordinates \[Back to [top](#toc)\]
$$\label{set_aphi}$$

We will first build the fundamental vector $A_i$ in spherical coordinates (see [Table 3](https://arxiv.org/pdf/1704.00599.pdf)). Note that we use reference_metric.py to set $r$ and $\theta$ in terms of Cartesian coordinates; this will save us a step later when we convert to Cartesian coordinates. So, we set 
\begin{align}
A_{\phi} &= \frac{\mu \varpi^2}{r^3}, \\
\end{align}
with $\mu = B_p R^3_{\rm NS} / 2$, $R_{\rm NS}$ is the neutron star radius, and $\varpi = \sqrt{x^2+y^2}$


In [2]:
r     = rfm.xxSph[0]
varpi = sp.sqrt(rfm.xxCart[0]**2 + rfm.xxCart[1]**2)

mu = B_p_aligned_rotator * R_NS_aligned_rotator**3 / 2
#mu = 5e-6

ASphD = ixp.zerorank1()

ASphD[2] = mu * varpi**2 / (r**3) # The other components were already declared to be 0.


<a id='jacobian'></a>

# Step 3: Use the Jacobian matrix to transform the vector to Cartesian coordinates \[Back to [top](#toc)\]
$$\label{jacobian}$$

Now, we will use the coordinate transformation definitions provided by reference_metric.py to build the Jacobian 
$$ 
\frac{\partial x_{\rm Sph}^j}{\partial x_{\rm Cart}^i},
$$ 
where $x_{\rm Sph}^j \in \{r,\theta,\phi\}$ and $x_{\rm Cart}^i \in \{x,y,z\}$. We would normally compute its inverse, but since none of the quantities we need to transform have upper indices, it is not necessary. Then, since $A_i$ and has one lower index, it will need to be multiplied by the Jacobian:

$$
A_i^{\rm Cart} = A_j^{\rm Sph} \frac{\partial x_{\rm Sph}^j}{\partial x_{\rm Cart}^i},
$$

$$\label{step3}$$

In [3]:
# Step 3: Use the Jacobian matrix to transform the vectors to Cartesian coordinates.
drrefmetric__dx_0UDmatrix = sp.Matrix([[sp.diff(rfm.xxSph[0],rfm.xx[0]), sp.diff(rfm.xxSph[0],rfm.xx[1]), sp.diff(rfm.xxSph[0],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[1],rfm.xx[0]), sp.diff(rfm.xxSph[1],rfm.xx[1]), sp.diff(rfm.xxSph[1],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[2],rfm.xx[0]), sp.diff(rfm.xxSph[2],rfm.xx[1]), sp.diff(rfm.xxSph[2],rfm.xx[2])]])
#dx__drrefmetric_0UDmatrix = drrefmetric__dx_0UDmatrix.inv() # We don't actually need this in this case.

AD = ixp.register_gridfunctions_for_single_rank1("AUX","AD")
for i in range(DIM):
    for j in range(DIM):
        AD[i] = drrefmetric__dx_0UDmatrix[(j,i)]*ASphD[j]
    print(AD[i])


-B_p_aligned_rotator*R_NS_aligned_rotator**3*xx1/(2*(xx0**2 + xx1**2 + xx2**2)**(3/2))
B_p_aligned_rotator*R_NS_aligned_rotator**3*xx0/(2*(xx0**2 + xx1**2 + xx2**2)**(3/2))
0


<a id='vi'></a>

# Step 4: Calculate $v^i$ \[Back to [top](#toc)\]
$$\label{vi}$$

Here, we will calculate the drift velocity $v^i = \Omega \textbf{e}_z \times \textbf{r} = [ijk] \Omega \textbf{e}^j_z x^k$, where $[ijk]$ is the Levi-Civita permutation symbol and $\textbf{e}^i_z = (0,0,1)$. Conveniently, in flat space, the drift velocity reduces to the Valencia velocity because $\alpha = 1$ and $\beta^i = 0$.


In [4]:
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaSymbolDDD = weyl.define_LeviCivitaSymbol_rank3()

ADdD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        ADdD[i][j] = sp.simplify(sp.diff(AD[i],rfm.xxCart[j]))
BU = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaSymbolDDD[i][j][k] * ADdD[k][j]

Omega_aligned_rotator = par.Cparameters("REAL",thismodule,"Omega_aligned_rotator") # The angular velocity of the NS
unit_zU = ixp.zerorank1()
unit_zU[2] = 1

ValenciavU = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            ValenciavU[i] += LeviCivitaSymbolDDD[i][j][k] * Omega_aligned_rotator * unit_zU[j] * rfm.xx[k]

<a id='code_validation'></a>

# Step 5: Code Validation against GiRaFFEfood_HO/GiRaFFEfood_HO_Aligned_Rotator NRPy+ Module  \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the $\texttt{GiRaFFE}$ Aligned Rotator initial data equations  we intend to use between

1. this tutorial and 
2. the NRPy+ [GiRaFFEfood_HO_Aligned_Rotator.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO_Aligned_Rotator.py) module.

In [5]:
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []

print("vvv Ignore the minor warnings below. vvv")

import GiRaFFEfood_HO.GiRaFFEfood_HO_Aligned_Rotator as gfho
gfho.GiRaFFEfood_HO_Aligned_Rotator()

print("^^^ Ignore the minor warnings above. ^^^\n")

print("Consistency check between GiRaFFEfood_HO tutorial and NRPy+ module: ALL SHOULD BE ZERO.")

for i in range(DIM):

    print("ValenciavU["+str(i)+"] - gfho.ValenciavU["+str(i)+"] = " + str(ValenciavU[i] - gfho.ValenciavU[i]))
    print("AD["+str(i)+"] - gfho.AD["+str(i)+"] = " + str(AD[i] - gfho.AD[i]))


vvv Ignore the minor warnings below. vvv
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::xmin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::xmax
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::ymin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::ymax
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::zmin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::zmax
initialize_param() minor warning: Did nothing; already initialized parameter GiRaFFEfood_HO_Aligned_Rotator::B_p_aligned_rotator
initialize_param() minor warning: Did nothing; already initialized parameter GiRaFFEfood_HO_Aligned_Rotator::R_NS_aligned_rotator


<a id='latex_pdf_output'></a>

# Step 6: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFEfood_HO_Aligned_Rotator.pdf](Tutorial-GiRaFFEfood_HO_Aligned_Rotator.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [6]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-GiRaFFEfood_HO_Aligned_Rotator.ipynb
!pdflatex -interaction=batchmode Tutorial-GiRaFFEfood_HO_Aligned_Rotator.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFEfood_HO_Aligned_Rotator.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFEfood_HO_Aligned_Rotator.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-GiRaFFEfood_HO_Aligned_Rotator.ipynb to latex
[NbConvertApp] Writing 41684 bytes to Tutorial-GiRaFFEfood_HO_Aligned_Rotator.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
